# xFLS master catalogue
## Preparation of DataFusion-Spitzer data

The catalogue comes from `dmu0_DataFusion-Spitzer`.

In the catalogue, we keep:

- The identifier (it's unique in the catalogue);
- The position;
- The stellarity;
- The aperture magnitude.
- The total magnitude.

We don't know when the maps have been observed. We will use the year of the reference paper.

In [1]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))

This notebook was run with herschelhelp_internal version: 
04829ed (Thu Nov 2 16:57:19 2017 +0000) [with local modifications]


In [2]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

from collections import OrderedDict
import os

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table
import numpy as np

from herschelhelp_internal.flagging import  gaia_flag_column
from herschelhelp_internal.masterlist import nb_astcor_diag_plot, remove_duplicates
from herschelhelp_internal.utils import astrometric_correction, flux_to_mag

/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/seaborn/apionly.py:6: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)


In [3]:
OUT_DIR =  os.environ.get('TMP_DIR', "./data_tmp")
try:
    os.makedirs(OUT_DIR)
except FileExistsError:
    pass

RA_COL = "spitzer_ra"
DEC_COL = "spitzer_dec"

## I - Column selection

In [4]:
imported_columns = OrderedDict({
        'id_1': "spitzer_id",
        'ra_1': "spitzer_ra",
        'dec_1': "spitzer_dec",
        'class_star_1':  "spitzer_stellarity", #Take IRAC1 stellarity
        'flux_auto_1': "f_irac_i1", 
        'fluxerr_auto_1': "ferr_irac_i1", 
        'flux_aper_2_1': "f_ap_irac_i1", #Is this 2 arcsec aperture
        'fluxerr_aper_2_1': "ferr_ap_irac_i1",
            'flux_auto_2': "f_irac_i2", 
        'fluxerr_auto_2': "ferr_irac_i2", 
        'flux_aper_2_2': "f_ap_irac_i2", 
        'fluxerr_aper_2_2': "ferr_ap_irac_i2",
            'flux_auto_3': "f_irac_i3", 
        'fluxerr_auto_3': "ferr_irac_i3", 
        'flux_aper_2_3': "f_ap_irac_i3", 
        'fluxerr_aper_2_3': "ferr_ap_irac_i3",
            'flux_auto_4': "f_irac_i4", 
        'fluxerr_auto_4': "ferr_irac_i4", 
        'flux_aper_2_4': "f_ap_irac_i4", 
        'fluxerr_aper_2_4': "ferr_ap_irac_i4",
    })


catalogue = Table.read("../../dmu0/dmu0_DataFusion-Spitzer/data/Datafusion-xFLS.fits")[list(imported_columns)]
for column in imported_columns:
    catalogue[column].name = imported_columns[column]

epoch = 2011

# Clean table metadata
catalogue.meta = None

In [5]:
# Adding flux and band-flag columns
for col in catalogue.colnames:
    if col.startswith('f_'):
        
        errcol = "ferr{}".format(col[1:])
        
       

        mag, error = flux_to_mag(np.array(catalogue[col]) * 1.e-6, np.array(catalogue[errcol]) * 1.e-6)
        
        # Fluxes are added in µJy
        catalogue.add_column(Column(mag , name="m{}".format(col[1:])))
        catalogue.add_column(Column(error , name="m{}".format(errcol[1:])))
        
        # Band-flag column
        if "ap" not in col:
            catalogue.add_column(Column(np.zeros(len(catalogue), dtype=bool), name="flag{}".format(col[1:])))
        
# TODO: Set to True the flag columns for fluxes that should not be used for SED fitting.

/Users/rs548/GitHub/herschelhelp_internal/herschelhelp_internal/utils.py:76: RuntimeWarning: invalid value encountered in log10
  magnitudes = 2.5 * (23 - np.log10(fluxes)) - 48.6


In [6]:
catalogue[:10].show_in_notebook()

idx,spitzer_id,spitzer_ra,spitzer_dec,spitzer_stellarity,f_irac_i1,ferr_irac_i1,f_ap_irac_i1,ferr_ap_irac_i1,f_irac_i2,ferr_irac_i2,f_ap_irac_i2,ferr_ap_irac_i2,f_irac_i3,ferr_irac_i3,f_ap_irac_i3,ferr_ap_irac_i3,f_irac_i4,ferr_irac_i4,f_ap_irac_i4,ferr_ap_irac_i4,m_irac_i1,merr_irac_i1,flag_irac_i1,m_ap_irac_i1,merr_ap_irac_i1,m_irac_i2,merr_irac_i2,flag_irac_i2,m_ap_irac_i2,merr_ap_irac_i2,m_irac_i3,merr_irac_i3,flag_irac_i3,m_ap_irac_i3,merr_ap_irac_i3,m_irac_i4,merr_irac_i4,flag_irac_i4,m_ap_irac_i4,merr_ap_irac_i4
,,deg,deg,,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,,,,,,,,,,,,,,,,,,,,
0,175248,260.0708529,59.5058729,0.01,7.529364237,4.84725015156,6.76117996957,2.69422897418,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,21.7081042328,0.698974683308,False,21.8249437596,0.432649619497,nan,nan,False,nan,nan,nan,nan,False,nan,nan,nan,nan,False,nan,nan
1,175647,260.0720138,59.5080576,0.96,8.2065651588,3.45625333584,10.3881421622,2.59785800755,8.0462637336,3.33812470356,10.4208714117,3.39632792263,24.1444028676,12.05729904,25.3070060644,11.9476938515,nan,nan,nan,nan,21.6145964448,0.45726553155,False,21.3586552892,0.271520012874,21.6360143415,0.450435502321,False,21.3552399076,0.353858717101,20.4429588274,0.542197964933,False,20.3918980771,0.512587057708,nan,nan,False,nan,nan
2,176015,260.072496,59.5111034,0.06,6.072903165,2.81363084148,6.28653120734,2.40644600136,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,21.9415091096,0.50303138193,False,21.9039723105,0.415613231256,nan,nan,False,nan,nan,nan,nan,False,nan,nan,nan,nan,False,nan,nan
3,176016,260.0684927,59.5107781,0.54,9.351565185,3.7007419476,9.56047410163,2.4708297388,12.578408979,3.81093504672,14.341344352,3.52339507173,nan,nan,nan,nan,nan,nan,nan,nan,21.4727892364,0.429663851718,False,21.4488014266,0.280600028272,21.1509357214,0.328950200388,False,21.0085253404,0.266744699741,nan,nan,False,nan,nan,nan,nan,False,nan,nan
4,176913,260.0726075,59.5162896,0.99,11.3777965902,2.60666052072,14.2421668679,2.83525274788,15.5058138264,5.41380173016,17.1187395486,4.34968938201,nan,nan,nan,nan,34.68808848,15.4381150572,46.1268808541,18.0070693039,21.2598545868,0.248742863209,False,21.0160598253,0.21614243019,20.9237635869,0.379081073049,False,20.8163205389,0.275874005098,nan,nan,False,nan,nan,20.0495490793,0.483212571961,False,19.7401147796,0.423851054369
5,178350,260.070973,59.5219171,0.53,42.335795256,4.71514921356,43.3140046223,3.99163352837,32.496407673,8.1111871308,28.2925142849,5.33290339156,32.5928924652,19.7023252128,38.5309007663,16.5219193149,nan,nan,nan,nan,19.8332306945,0.120923870239,False,19.8084291536,0.100056807854,20.120411614,0.271002555732,False,20.2708211404,0.20465223523,20.1171927406,0.656324928028,False,19.9354770951,0.465559995107,nan,nan,False,nan,nan
6,177594,260.0678615,59.5182518,0.0,9.4890950214,4.78143254304,6.69153814565,2.86682425989,4.2629476998,3.7817794254,6.62973651704,4.07644343883,nan,nan,nan,nan,nan,nan,nan,nan,21.4569380111,0.547088464272,False,21.8361851049,0.465156862875,22.3257249887,0.963186773499,False,21.8462593281,0.66758946103,nan,nan,False,nan,nan,nan,nan,False,nan,nan
7,179493,260.0587367,59.5334548,0.96,27.2595954768,5.09399629152,29.3522307106,4.3063599406,17.2509575862,5.97194257776,18.0575784268,4.8318351062,nan,nan,nan,nan,nan,nan,nan,nan,20.3112014831,0.20289135271,False,20.2308972191,0.159291842052,20.8079669815,0.37586054206,False,20.7583512256,0.290520588432,nan,nan,False,nan,nan,nan,nan,False,nan,nan
8,179972,260.0618242,59.5345495,0.02,22.4085261426,5.77371603264,21.0770400644,4.19698401636,11.1922562028,4.16206495836,14.2803790081,4.72672058615,nan,nan,nan,nan,nan,nan,nan,nan,20.5239667675,0.279747650191,False,20.5904759474,0.216198170304,21.2777058923,0.403752784958,False,21.0131506651,0.359372231454,nan,nan,False,nan,nan,nan,nan,False,nan,nan


## II - Removal of duplicated sources

We remove duplicated objects from the input catalogues.

In [7]:
SORT_COLS = ['ferr_ap_irac_i1', 'ferr_ap_irac_i2', 'ferr_ap_irac_i3', 'ferr_ap_irac_i4']
FLAG_NAME = 'spitzer_flag_cleaned'

nb_orig_sources = len(catalogue)

catalogue = remove_duplicates(catalogue, RA_COL, DEC_COL, sort_col=SORT_COLS,flag_name=FLAG_NAME)

nb_sources = len(catalogue)

print("The initial catalogue had {} sources.".format(nb_orig_sources))
print("The cleaned catalogue has {} sources ({} removed).".format(nb_sources, nb_orig_sources - nb_sources))
print("The cleaned catalogue has {} sources flagged as having been cleaned".format(np.sum(catalogue[FLAG_NAME])))

/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/astropy/coordinates/angles.py:643: RuntimeWarning: invalid value encountered in less
  if np.any(self_angle < wrap_angle_floor) or np.any(self_angle >= wrap_angle):
/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/astropy/coordinates/angles.py:643: RuntimeWarning: invalid value encountered in greater_equal
  if np.any(self_angle < wrap_angle_floor) or np.any(self_angle >= wrap_angle):
/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/astropy/coordinates/angles.py:526: RuntimeWarning: invalid value encountered in less
  if np.any(angles.value < lower) or np.any(angles.value > upper):
/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/astropy/coordinates/angles.py:526: RuntimeWarning: invalid value encountered in greater
  if np.any(angles.value < lower) or np.any(angles.value > upper):


The initial catalogue had 28621 sources.
The cleaned catalogue has 28621 sources (0 removed).
The cleaned catalogue has 0 sources flagged as having been cleaned


## III - Astrometry correction

We match the astrometry to the Gaia one. We limit the Gaia catalogue to sources with a g band flux between the 30th and the 70th percentile. Some quick tests show that this give the lower dispersion in the results.

In [8]:
gaia = Table.read("../../dmu0/dmu0_GAIA/data/GAIA_xFLS.fits")
gaia_coords = SkyCoord(gaia['ra'], gaia['dec'])

In [9]:
nb_astcor_diag_plot(catalogue[RA_COL], catalogue[DEC_COL], 
                    gaia_coords.ra, gaia_coords.dec)

/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/astropy/coordinates/angles.py:643: RuntimeWarning: invalid value encountered in less
  if np.any(self_angle < wrap_angle_floor) or np.any(self_angle >= wrap_angle):
/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/astropy/coordinates/angles.py:643: RuntimeWarning: invalid value encountered in greater_equal
  if np.any(self_angle < wrap_angle_floor) or np.any(self_angle >= wrap_angle):
/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/astropy/coordinates/angles.py:526: RuntimeWarning: invalid value encountered in less
  if np.any(angles.value < lower) or np.any(angles.value > upper):
/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/astropy/coordinates/angles.py:526: RuntimeWarning: invalid value encountered in greater
  if np.any(angles.value < lower) or np.any(angles.value > upper):


IndexError: index 39625 is out of bounds for axis 1 with size 39625

In [ ]:
delta_ra, delta_dec =  astrometric_correction(
    SkyCoord(catalogue[RA_COL], catalogue[DEC_COL]),
    gaia_coords
)

print("RA correction: {}".format(delta_ra))
print("Dec correction: {}".format(delta_dec))

In [ ]:
catalogue[RA_COL] +=  delta_ra.to(u.deg)
catalogue[DEC_COL] += delta_dec.to(u.deg)

In [ ]:
nb_astcor_diag_plot(catalogue[RA_COL], catalogue[DEC_COL], 
                    gaia_coords.ra, gaia_coords.dec)

## IV - Flagging Gaia objects

In [ ]:
catalogue.add_column(
    gaia_flag_column(SkyCoord(catalogue[RA_COL], catalogue[DEC_COL]), epoch, gaia)
)

In [ ]:
GAIA_FLAG_NAME = "spitzer_flag_gaia"

catalogue['flag_gaia'].name = GAIA_FLAG_NAME
print("{} sources flagged.".format(np.sum(catalogue[GAIA_FLAG_NAME] > 0)))

# V - Saving to disk

In [ ]:
catalogue.write("{}/Spitzer.fits".format(OUT_DIR), overwrite=True)